<a href="https://colab.research.google.com/github/mayankbrn/9.7_Delhivery_feature_engineering/blob/MA_working/07_Delhivery_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Case: Delhivery - Feature Engineering



## About Delhivery

Delhivery is the largest and fastest-growing fully integrated player in India by revenue in Fiscal 2021. They aim to build the operating system for commerce, through a combination of world-class infrastructure, logistics operations of the highest quality, and cutting-edge engineering and technology capabilities.

The Data team builds intelligence and capabilities using this data that helps them to widen the gap between the quality, efficiency, and profitability of their business versus their competitors.

### How can you help here?



The company wants to understand and process the data coming out of data engineering pipelines:

• Clean, sanitize and manipulate data to get useful features out of raw fields

• Make sense out of the raw data and help the data science team to build forecasting models on it

### Dataset

Dataset Link: [Delhivery data](https://drive.google.com/file/d/1ZkF2gGCDkjwQgOTGVBpsqhPpSGg1Fybb/view?usp=drive_link)



### Column Profiling

The dataset at the heart of this exploration. Here are some of the key features:

- **data**: tells whether the data is testing or training data.
- **trip_creation_time**: Timestamp of trip creation.
- **route_schedule_uuid**: Unique Id for a particular route schedule.
- **route_type**: Transportation type.
- **FTL**: Full Truck Load - FTL shipments get to the destination sooner, as the truck is making no other pickups or drop-offs along the way.
- **Carting**: Handling system consisting of small vehicles (carts).
- **trip_uuid**: Unique ID given to a particular trip (A trip may include different source and destination centers).
- **source_center**: Source ID of trip origin.
- **source_name**: Source Name of trip origin.
- **destination_center**: Destination ID.
- **destination_name**: Destination Name.
- **od_start_time**: Trip start time.
- **od_end_time**: Trip end time.
- **start_scan_to_end_scan**: Time taken to deliver from source to destination.
- **is_cutoff**: Unknown field.
- **cutoff_factor**: Unknown field.
- **cutoff_timestamp**: Unknown field.
- **actual_distance_to_destination**: Distance in Kms between source and destination warehouse.
- **actual_time**: Actual time taken to complete the delivery (Cumulative).
- **osrm_time**: An open-source routing engine time calculator which computes the shortest path between points in a given map (Includes usual traffic, distance through major and minor roads) and gives the time (Cumulative).
- **osrm_distance**: An open-source routing engine which computes the shortest path between points in a given map (Includes usual traffic, distance through major and minor roads) (Cumulative).
- **factor**: Unknown field.
- **segment_actual_time**: This is a segment time. Time taken by the subset of the package delivery.
- **segment_osrm_time**: This is the OSRM segment time. Time taken by the subset of the package delivery.
- **segment_osrm_distance**: This is the OSRM distance. Distance covered by the subset of the package delivery.
- **segment_factor**: Unknown field.

### Concepts Used

- Feature Creation
- Relationship between Features
- Column Normalization /Column Standardization
- Handling categorical values
- Missing values - Outlier treatment / Types of outliers

### How to begin:

Since delivery details of one package are divided into several rows (think of it as connecting flights to reach a particular destination). Now think about how we should treat their fields if we combine these rows? What aggregation would make sense if we merge. What would happen to the numeric fields if we merge the rows?

#### Hint:


You can use inbuilt functions like `groupby` and aggregations like `sum()`, `cumsum()` to merge some rows based on their
1. `Trip_uuid`, `Source ID` and `Destination ID`
2. Further aggregate on the basis of just `Trip_uuid`. You can also keep the first and last values for some numeric/categorical fields if aggregating them won’t make sense.

#### Basic data cleaning and exploration:

- Handle missing values in the data.
- Analyze the structure of the data.
- Try merging the rows using the hint mentioned above.
- Build some features to prepare the data for actual analysis. Extract features from the below fields:
  - **Destination Name**: Split and extract features out of destination. City-place-code (State)
  - **Source Name**: Split and extract features out of destination. City-place-code (State)
  - **Trip_creation_time**: Extract features like month, year and day etc

#### In-depth analysis and feature engineering:

- Calculate the time taken between `od_start_time` and `od_end_time` and keep it as a feature. Drop the original columns, if required
- Compare the difference between Point a. and `start_scan_to_end_scan`. Do hypothesis testing/ Visual analysis to check.
- Do hypothesis testing/ visual analysis between `actual_time` aggregated value and `OSRM time` aggregated value (aggregated values are the values you’ll get after merging the rows on the basis of `trip_uuid`)
- Do hypothesis testing/ visual analysis between `actual_time` aggregated value and `segment actual time` aggregated value (aggregated values are the values you’ll get after merging the rows on the basis of `trip_uuid`)
- Do hypothesis testing/ visual analysis between `osrm distance` aggregated value and `segment osrm distance` aggregated value (aggregated values are the values you’ll get after merging the rows on the basis of `trip_uuid`)
- Do hypothesis testing/ visual analysis between `osrm time` aggregated value and `segment osrm time` aggregated value (aggregated values are the values you’ll get after merging the rows on the basis of `trip_uuid`)
- Find outliers in the numerical variables (you might find outliers in almost all the variables), and check it using visual analysis
- Handle the outliers using the **IQR method**.
- Do one-hot encoding of categorical variables (like `route_type`)
- Normalize/ Standardize the numerical features using `MinMaxScaler` or `StandardScaler`.

### Evaluation Criteria (100 Points):


- Define Problem Statement and Perform Exploratory Data Analysis (10 points)

  - **Definition of Problem** (as per given problem statement with additional views)
  - **Observations** on:
    - Shape of data
    - Data types of all the attributes
    - Conversion of categorical attributes to 'category' (if required)
    - Missing value detection
    - Statistical summary
  - **Visual Analysis**:
    - Distribution plots of all the continuous variable(s)
    - Boxplots of all the categorical variables
  - **Insights** based on EDA
  - **Comments** on:
    - Range of attributes
    - Outliers of various attributes
    - Distribution of the variables and relationship between them
  - **Comments** for each univariate and bivariate plot

- Feature Creation (10 Points)
- Merging of Rows and Aggregation of Fields (10 Points)
- Comparison & Visualization of Time and Distance Fields (10 Points)
- Missing Values Treatment & Outlier Treatment (10 Points)
- Checking Relationship Between Aggregated Fields (10 Points)
- Handling Categorical Values (10 Points)
- Column Normalization / Column Standardization (10 Points)

- Business Insights (10 Points):
  Should include patterns observed in the data along with what you can infer from it. Examples:
  - Check from where most orders are coming from (State, Corridor, etc.)
  - Busiest corridor, average distance between them, average time taken

- Recommendations (10 Points)
  Actionable items for business. No technical jargon. No complications. Simple action items that everyone can understand.

## Solution

### Basic data cleaning and exploration:

In [1]:
#importing the relevent libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#importing the csv file from google drive
!gdown 1ZkF2gGCDkjwQgOTGVBpsqhPpSGg1Fybb

Downloading...
From: https://drive.google.com/uc?id=1ZkF2gGCDkjwQgOTGVBpsqhPpSGg1Fybb
To: /content/07_delhivery_data.csv
100% 55.6M/55.6M [00:01<00:00, 39.0MB/s]


In [3]:
#load the csv into dataframe
df = pd.read_csv('07_delhivery_data.csv')

In [4]:
df.head(5)

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
0,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:27:55,10.435660,14.0,11.0,11.9653,1.272727,14.0,11.0,11.9653,1.272727
1,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:17:55,18.936842,24.0,20.0,21.7243,1.200000,10.0,9.0,9.7590,1.111111
2,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:01:19.505586,27.637279,40.0,28.0,32.5395,1.428571,16.0,7.0,10.8152,2.285714
3,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:39:57,36.118028,62.0,40.0,45.5620,1.550000,21.0,12.0,13.0224,1.750000
4,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:33:55,39.386040,68.0,44.0,54.2181,1.545455,6.0,5.0,3.9153,1.200000


#### Analyzing the structure of data

In [8]:
#shape of data
df.shape

(144867, 24)

There are 24 columns and 144867 rows in the dataset.

In [5]:
#checking the datatype of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144867 entries, 0 to 144866
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   data                            144867 non-null  object 
 1   trip_creation_time              144867 non-null  object 
 2   route_schedule_uuid             144867 non-null  object 
 3   route_type                      144867 non-null  object 
 4   trip_uuid                       144867 non-null  object 
 5   source_center                   144867 non-null  object 
 6   source_name                     144574 non-null  object 
 7   destination_center              144867 non-null  object 
 8   destination_name                144606 non-null  object 
 9   od_start_time                   144867 non-null  object 
 10  od_end_time                     144867 non-null  object 
 11  start_scan_to_end_scan          144867 non-null  float64
 12  is_cutoff       

We can see that time is considered as object and float in no of columns

In [6]:
#checking Null values in the columns
df.isna().sum()

,0
data,0
trip_creation_time,0
route_schedule_uuid,0
route_type,0
trip_uuid,0
source_center,0
source_name,293
destination_center,0
destination_name,261
od_start_time,0


'Source name' and 'Destination name' have some missing values.

In [9]:
#unique values in the columns
df.nunique()

,0
data,2
trip_creation_time,14817
route_schedule_uuid,1504
route_type,2
trip_uuid,14817
source_center,1508
source_name,1498
destination_center,1481
destination_name,1468
od_start_time,26369


In [16]:
# statistical summary of data - Numercial columns
df.describe()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
count,144867.000000,144867.000000,144867.000000,144867.000000,144867.000000,144867.000000,144867.000000,144867.000000,144867.000000,144867.00000,144867.000000
mean,961.262986,232.926567,234.073372,416.927527,213.868272,284.771297,2.120107,36.196111,18.507548,22.82902,2.218368
std,1037.012769,344.755577,344.990009,598.103621,308.011085,421.119294,1.715421,53.571158,14.775960,17.86066,4.847530
min,20.000000,9.000000,9.000045,9.000000,6.000000,9.008200,0.144000,-244.000000,0.000000,0.00000,-23.444444
25%,161.000000,22.000000,23.355874,51.000000,27.000000,29.914700,1.604264,20.000000,11.000000,12.07010,1.347826
50%,449.000000,66.000000,66.126571,132.000000,64.000000,78.525800,1.857143,29.000000,17.000000,23.51300,1.684211
75%,1634.000000,286.000000,286.708875,513.000000,257.000000,343.193250,2.213483,40.000000,22.000000,27.81325,2.250000
max,7898.000000,1927.000000,1927.447705,4532.000000,1686.000000,2326.199100,77.387097,3051.000000,1611.000000,2191.40370,574.250000


In [18]:
#statistical summary of data - categorical columns
df.describe(include = object)

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,cutoff_timestamp
count,144867,144867,144867,144867,144867,144867,144574,144867,144606,144867,144867,144867
unique,2,14817,1504,2,14817,1508,1498,1481,1468,26369,26369,93180
top,training,2018-09-28 05:23:15.359220,thanos::sroute:4029a8a2-6c74-4b7e-a6d8-f9e069f...,FTL,trip-153811219535896559,IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),2018-09-21 18:37:09.322207,2018-09-24 09:59:15.691618,2018-09-24 05:19:20
freq,104858,101,1812,99660,101,23347,23347,15192,15192,81,81,40


#### Handling missing values and remove incorrect data

In [25]:
# checking for the source_center  for which source_name is null

df[df['source_name'].isna()]['source_center'].unique()

array(['IND342902A1B', 'IND577116AAA', 'IND282002AAD', 'IND465333A1B',
       'IND841301AAC', 'IND509103AAC', 'IND126116AAA', 'IND331022A1B',
       'IND505326AAB', 'IND852118A1B'], dtype=object)

This give us all the source_center names for which source_name is not available.
Let's check if these source_center name is having source_name

In [39]:
# Checking if we can get the source_name for above source_centers from other rows in the data
# In short, it's selecting rows where "source_name" has a value, but the "source_center" is associated with some other rows where "source_name" is missing (null).

df[(df['source_name'].notnull()) & (df['source_center'].isin(df[df['source_name'].isnull()]))]

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor


It means that source_name for missing values is not available in other rows as well.

In [32]:
#checking for destination_center for which destination_name is null
df[df['destination_name'].isna()]['destination_center'].unique()

array(['IND342902A1B', 'IND577116AAA', 'IND282002AAD', 'IND465333A1B',
       'IND841301AAC', 'IND505326AAB', 'IND852118A1B', 'IND126116AAA',
       'IND509103AAC', 'IND221005A1A', 'IND250002AAC', 'IND331001A1C',
       'IND122015AAC'], dtype=object)

In [38]:
# Checking if we can get the destintion_name for above destinatio_centers from other rows in the data
# In short, it's selecting rows where "destination_name" has a value, but the "destination_center" is associated with some other rows where "destination_name" is missing (null).

df[(df['destination_name'].notnull()) & (df['destination_center'].isin(df[df['destination_name'].isnull()]))]

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor


It means that destinatio_name for missing values is not available in other rows as well.

In [43]:
# We can see from the describe that the values of segment actual time is negative, which can't be true so lets drop that data
df.drop(df[df['segment_actual_time']<0].index, inplace = True)

In [44]:
#now checking the describe data
df.describe()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
count,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000,144846.000000
mean,961.226537,232.911057,234.057171,416.908724,213.853002,284.750969,2.120190,36.207427,18.507304,22.828528,2.219084
std,1036.993595,344.740981,344.974984,598.085058,307.997702,421.101831,1.715508,53.561259,14.775870,17.860268,4.847144
min,20.000000,9.000000,9.000045,9.000000,6.000000,9.008200,0.144000,0.000000,0.000000,0.000000,-1.000000
25%,161.000000,22.000000,23.354927,51.000000,27.000000,29.909925,1.604288,20.000000,11.000000,12.070100,1.347826
50%,449.000000,66.000000,66.126234,132.000000,64.000000,78.524600,1.857143,29.000000,17.000000,23.513000,1.684211
75%,1634.000000,286.000000,286.706673,513.000000,257.000000,343.062075,2.213589,40.000000,22.000000,27.812975,2.250000
max,7898.000000,1927.000000,1927.447705,4532.000000,1686.000000,2326.199100,77.387097,3051.000000,1611.000000,2191.403700,574.250000


Now negative values of negative segment time are removed.

#### Univariate Analysis

#### Bi-variate Analysis

#### Merging rows of data

### Feature Generation

### Hypothesis testing

### Outlier Detection

### Outlier Treatment

### Encoding

### Normalisation/ Standardization

## Business Insights

### Recommendations